In [1]:
from pathlib import Path
import numpy as np, pandas as pd, re

FPATH_SBIC = Path('./')

train_raw = pd.read_csv(FPATH_SBIC/'SBIC.v2.agg.trn.csv')

## Original Data Import

In [2]:
train_raw

,Unnamed: 0,post,targetMinority,targetCategory,targetStereotype,whoTarget,intentYN,sexYN,offensiveYN,dataSource,hasBiasedImplication
0,0,"\n\nBill Kristol and Ben Shaprio, two turds in...",[],[],[],0.0,0.886667,0.0,1.000000,Gab,1
1,1,\n\nRose\n🌹Taylor‏ @RealRoseTaylor 6h6 hours a...,[],[],[],NaN,0.220000,0.0,0.000000,Gab,1
2,2,\nCharlie Kirk‏\n\nJohnny Depp calls for death...,"[""liberals""]","[""social""]","[""miscontrue things"", ""take things in a weird ...",1.0,1.000000,0.0,1.000000,Gab,0
3,3,\nDavid Knight‏ \n\nNotice how quickly things ...,[],[],[],0.0,0.110000,0.0,0.166667,Gab,1
4,4,\nFinland fireball: Time-lapse video shows nig...,[],[],[],NaN,0.000000,0.0,0.000000,Gab,1
...,...,...,...,...,...,...,...,...,...,...,...
35419,35498,👉 Illegally in the country after 5 deportation...,[],[],[],NaN,0.220000,0.0,0.000000,Gab,1
35420,35500,💥Breaking💥\nJulian Assange is the gate keeper ...,[],[],[],0.0,0.000000,0.0,0.000000,Gab,1
35421,35501,📖 2Kings 22:19 because your heart was peniten...,[],[],[],NaN,0.000000,0.0,0.000000,Gab,1
35422,35502,🚨#FAKENEWSAWARDS🚨\n\n🚨 who is #1 fake news ?🚨\...,[],[],[],0.0,0.553333,0.0,0.500000,Gab,1


## Filter for Target Classes

In [3]:
targets = ['women', 'black', 'asian', 'jewish', 'muslim', 'gay', 'lesbian', 'trans', 'bisexual', 'asexual']

target_minority = []
def has_class(targetMinority):
    # return len(  set(targets).intersection( set(eval(targetMinority)) )  ) > 0
    for target in targets:
        if target in targetMinority.lower():
            target = 'LGBT' if target in ['gay', 'lesbian', 'trans', 'bisexual', 'asexual'] else target
            target_minority.append(target)
            return True
    return False

train_filtered = train_raw[ train_raw.targetMinority.apply(has_class) ]
train_preproc = pd.DataFrame()
train_preproc['post'] = train_filtered.post
train_preproc['target_minority'] = target_minority
train_preproc = train_preproc.reset_index(drop=True)

print(train_preproc.target_minority.value_counts())
train_preproc

women     3129
black     2823
jewish     931
muslim     474
LGBT       283
asian      264
Name: target_minority, dtype: int64


,post,target_minority
0,\nJust watched facial recognition technology f...,muslim
1,I propose a new law \n\n Any congressman or w...,women
2,!!! RT @mayasolovely: As a woman you shouldn't...,women
3,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,women
4,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,women
...,...,...
7899,“Woman is not made to be the admiration of all...,women
7900,"“You can drink an ugly woman pretty, faster th...",women
7901,"“You da bomb!” “No, you da bomb!” In America –...",muslim
7902,⁠Why do wh*te people like to play hockey? It’...,black


## Pre-Process Data + Add Integer Labels

In [4]:
def textproc(text):
    urls = r'https*://.+?[\s\n]*$'              # remove urls
    mentions = r'@.+?\s'                        # remove @mentions
    misc = r'&#\d+;|&.+?;'                      # remove miscellaneous items such as &entities
    chars = r'[\r\n]'                           # remove line breaks
    exclude = r'[^a-zA-Z0-9\s\'\.\,\;\!\-\@\#]' # keep these characters
    processed = re.sub(urls, 'http ', text).strip()
    processed = re.sub(mentions, '@user ', processed).strip()
    processed = re.sub(r'(%s|%s|%s)'% (misc, chars, exclude), '', processed).strip()
    processed = np.NaN if len(processed.replace(' ', ''))==0 else processed
    return processed

classes = list(train_preproc.target_minority.unique())
labels = [classes.index(c) for c in train_preproc.target_minority]

train_postproc = pd.DataFrame()
train_postproc = train_preproc.copy()

train_postproc.post = train_postproc.post.apply(textproc)
train_postproc['labels'] = labels
train_postproc = train_postproc.dropna()


print(classes)
train_postproc

['muslim', 'women', 'black', 'LGBT', 'jewish', 'asian']


,post,target_minority,labels
0,Just watched facial recognition technology fro...,muslim,0
1,I propose a new law Any congressman or woman ...,women,1
2,!!! RT @user As a woman you shouldn't complain...,women,1
3,!!!!!!! RT @user Dawg!!!! RT @user You ever fu...,women,1
4,!!!!!!!!! RT @user @user she look like a tranny,women,1
...,...,...,...
7898,Post your dark jokes here! No joke is too dark...,black,2
7899,"Woman is not made to be the admiration of all,...",women,1
7900,"You can drink an ugly woman pretty, faster tha...",women,1
7901,"You da bomb! No, you da bomb! In America a co...",muslim,0


## Output to Required Format
Uncomment to save/overwrite the existing data

In [5]:
# train_postproc.post.to_csv('dataset.txt', header=None, index=None)
# train_postproc.labels.to_csv('labels.txt', header=None, index=None)
# np.savetxt('classes.txt', classes, fmt='%s')

## Compare Pre-Proc and Post-Proc (Check Processing)
Below we show the removal of @mentions, &entities, links, linebreak characters, linefeed characters, emojis as well as blank(NaN) lines

### Pre-Processed Posts

In [6]:
pd.set_option('max_colwidth', 150)
print('Total rows in PRE-processed DataFrame: %s' % len(train_preproc))
train_preproc.loc[[1, 13, 14, 19, 39, 180, 3290, 4552, 3282, 4518], :]

Total rows in PRE-processed DataFrame: 7904


,post,target_minority
1,I propose a new law \n\n Any congressman or woman that votes for DACA or any kind of immigration from Third World countries first has to house a ...,women
13,"""#BlackMonday: """"Hey, black people, we'll show solidarity with you, but not about shit that actually matters.""""""",black
14,"""&#128077;&#127997;&#128178;&#128176;&#128076;&#127996;&#128664;&#128663;&#127758;&#128175;&#127808;&#128013;&#127794;™&#128064;&#128077;&#127997;...",black
19,"""@Bill_ee_O @LFBarfe @Trubshawe1 @bwelliott13 """"And call the bus driver 'darkie' to impress the girls""""""",black
39,"""@ozzie_garza: @white_thunduh I'm about to cool faggot chill"" tf is cool nigga",black
180,&#8220;@Mojo4Life_: We'll I'm going to sleep &#128530;&#8221; fag,LGBT
3290,Nasty schoolgirl Avril Hall gets fucked https://t.co/RKvmW6NdPi,women
4552,We can't trust these hoes fam http://t.co/INvr9Wnccv,women
3282,NI🅱️🅱️ERS Whats the diffrence between pizza and a nigger?\n\n\n\n\n\n-PIZZA CAN FEED THE FAMILY,black
4518,WE TRUMP SUPPORTERS ARE THE 🔨 OF THOR THEM ENEMIES ARE IN DISSARAY SEND THEM MUSLIM INVADERS TO THEIR HALLOWED GRAVES DON'T SPARE A SINGLE ONE,muslim


### Post-Processed Posts

In [7]:
print('Total rows in POST-processed DataFrame: %s' % len(train_postproc))
train_postproc.loc[[1, 13, 14, 19, 39, 180, 3290, 4552, 3282, 4518], :]

Total rows in POST-processed DataFrame: 7903


,post,target_minority,labels
1,I propose a new law Any congressman or woman that votes for DACA or any kind of immigration from Third World countries first has to house a famil...,women,1
13,"#BlackMonday Hey, black people, we'll show solidarity with you, but not about shit that actually matters.",black,2
14,"Gang Gang Hooligan - The Real 1, Cracka Nigga That Don't Believe In Rapest, ya 3g Wana Si, bitxh ya out ya F",black,2
19,@user @user @user @user And call the bus driver 'darkie' to impress the girls,black,2
39,@user @user I'm about to cool faggot chill tf is cool nigga,black,2
180,@user We'll I'm going to sleep fag,LGBT,3
3290,Nasty schoolgirl Avril Hall gets fucked http,women,1
4552,We can't trust these hoes fam http,women,1
3282,NIERS Whats the diffrence between pizza and a nigger-PIZZA CAN FEED THE FAMILY,black,2
4518,WE TRUMP SUPPORTERS ARE THE OF THOR THEM ENEMIES ARE IN DISSARAY SEND THEM MUSLIM INVADERS TO THEIR HALLOWED GRAVES DON'T SPARE A SINGLE ONE,muslim,0
